In [3]:
import yaml
from rosbag.bag import Bag

In [4]:
bag1_filename = 'rosbags/record_2024_03_27_10_51_34.bag'
bag2_filename = 'rosbags/record_2024_03_27_11_03_09.bag'

In [5]:
bag = Bag(bag1_filename, 'r')
bag_info = yaml.load(bag._get_yaml_info(), Loader=yaml.Loader)
with open('bag_info.yaml', 'w') as file:
    yaml.dump(bag_info, file)

In [6]:
topics = bag.get_type_and_topic_info()[1].keys()
types = [val[0] for val in bag.get_type_and_topic_info()[1].values()]

print('Topics:', topics)
print('Types:', types)

Topics: dict_keys(['/audio', '/joint_states', '/xtion/rgb/image_raw_throttled'])
Types: ['audio_common_msgs/AudioData', 'sensor_msgs/JointState', 'sensor_msgs/Image']


In [7]:
import cv2
from cv_bridge import CvBridge

bridge = CvBridge()

for i, (_, message, t) in enumerate(bag.read_messages(topics='/xtion/rgb/image_raw_throttled')):
    cv2_image = bridge.imgmsg_to_cv2(message, desired_encoding='passthrough')
    cv2_image = cv2.cvtColor(cv2_image, cv2.COLOR_BGR2RGB)
    cv2.imwrite(f'images/frame{i:04d}.jpg', cv2_image)

In [8]:
# Extract video
def get_fps():
    for topic in bag_info['topics']:
        if topic['type'] == 'sensor_msgs/Image':
            return topic['frequency']
    return None

import subprocess
subprocess.run(['ffmpeg', '-r', str(get_fps()), '-i', r'images/frame%04d.jpg', 'video.avi'])

ffmpeg version 6.1.1 Copyright (c) 2000-2023 the FFmpeg developers
  built with clang version 16.0.6
  configuration: --prefix=/Users/runner/miniforge3/conda-bld/ffmpeg_1710227269443/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_pl --cc=arm64-apple-darwin20.0.0-clang --cxx=arm64-apple-darwin20.0.0-clang++ --nm=arm64-apple-darwin20.0.0-nm --ar=arm64-apple-darwin20.0.0-ar --disable-doc --disable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libharfbuzz --enable-libfontconfig --enable-libopenh264 --enable-libdav1d --enable-cross-compile --arch=arm64 --target-os=darwin --cross-prefix=arm64-apple-darwin20.0.0- --host-cc=/Users/runner/miniforge3/conda-bld/ffmpeg_1710227269443/_build_env/bin/x86_64-apple-darwin13.4.0-clang --enable-neon --enable-gnutls --enable-libmp3lame --enable-libvpx --enable-libass -

CompletedProcess(args=['ffmpeg', '-r', '3.7532', '-i', 'images/frame%04d.jpg', 'video.avi'], returncode=0)

In [9]:
# Extract audio
with open('audio.mp3', 'wb') as f:
    for i, (_, message, t) in enumerate(bag.read_messages(topics='/audio')):
        for byte in message.data:
            f.write(int.to_bytes(byte))

In [10]:
# Find audio delay
first_video_timestamp_ns = next(bag.read_messages(topics='/xtion/rgb/image_raw_throttled')).timestamp
first_audio_timestamp_ns = next(bag.read_messages(topics='/audio')).timestamp
audio_delay_in_seconds = (first_audio_timestamp_ns - first_video_timestamp_ns) / 1000000000
print(audio_delay_in_seconds)

0


In [11]:
# Combine video and audio
subprocess.run(['ffmpeg', '-i', 'video.avi', '-i', 'audio.mp3', '-c:v', 'copy', '-map', '0:v', '-map', '1:a', '-y', 'output.mp4'])

ffmpeg version 6.1.1 Copyright (c) 2000-2023 the FFmpeg developers
  built with clang version 16.0.6
  configuration: --prefix=/Users/runner/miniforge3/conda-bld/ffmpeg_1710227269443/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_pl --cc=arm64-apple-darwin20.0.0-clang --cxx=arm64-apple-darwin20.0.0-clang++ --nm=arm64-apple-darwin20.0.0-nm --ar=arm64-apple-darwin20.0.0-ar --disable-doc --disable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libharfbuzz --enable-libfontconfig --enable-libopenh264 --enable-libdav1d --enable-cross-compile --arch=arm64 --target-os=darwin --cross-prefix=arm64-apple-darwin20.0.0- --host-cc=/Users/runner/miniforge3/conda-bld/ffmpeg_1710227269443/_build_env/bin/x86_64-apple-darwin13.4.0-clang --enable-neon --enable-gnutls --enable-libmp3lame --enable-libvpx --enable-libass -

CompletedProcess(args=['ffmpeg', '-i', 'video.avi', '-i', 'audio.mp3', '-c:v', 'copy', '-map', '0:v', '-map', '1:a', '-y', 'output.mp4'], returncode=0)

In [10]:
from google.cloud import speech_v1p1beta1 as speech
import io
import os

def generate_srt(uri):
    client = speech.SpeechClient()

    audio = speech.RecognitionAudio(uri=uri)
    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.MP3,
        sample_rate_hertz=16000,
        language_code='en-US',
        enable_automatic_punctuation=True,
        enable_word_time_offsets=True,
        enable_speaker_diarization=True,
        diarization_speaker_count=2,  
    )

    operation = client.long_running_recognize(config=config, audio=audio)
    print("Waiting for operation to complete...")
    response = operation.result(timeout=90)

    srt_file_name = os.path.basename(uri).rsplit(".", 1)[0] + ".srt"
    with open(srt_file_name, 'w') as srt_file:
        counter = 1
        for result in response.results:
            if result.alternatives[0].words:
                start_time = result.alternatives[0].words[0].start_time
                end_time = result.alternatives[0].words[-1].end_time
                speaker_tag = result.alternatives[0].words[0].speaker_tag

                srt_file.write(f"{counter}\n")
                srt_file.write(f"{format_time(start_time)} --> {format_time(end_time)}\n")
                srt_file.write(f"Speaker {speaker_tag}: {result.alternatives[0].transcript}\n\n")
                counter += 1
    print(f"SRT file created: {srt_file_name}")

def format_time(duration):
    total_seconds = int(duration.total_seconds())
    hours = total_seconds // 3600
    minutes = (total_seconds % 3600) // 60
    seconds = total_seconds % 60
    microseconds = duration.microseconds
    milliseconds = microseconds // 1000

    return f"{hours:02}:{minutes:02}:{seconds:02},{milliseconds:03}"

generate_srt("gs://comp90082-audio-bucket/audio.mp3")



Waiting for operation to complete...
SRT file created: audio.srt


In [2]:
import whisper
import datetime

model = whisper.load_model("base")

result = model.transcribe("audio.mp3")

def seconds_to_srt_time(seconds):
    return str(datetime.timedelta(seconds=seconds))

srt_content = ""
for i, segment in enumerate(result["segments"]):
    start = seconds_to_srt_time(segment["start"])
    end = seconds_to_srt_time(segment["end"])
    text = segment["text"]
    srt_content += f"{i+1}\n{start} --> {end}\n{text}\n\n"
    
with open("output3.srt", "w") as file:
    file.write(srt_content)


/Users/ethan/anaconda3/lib/python3.11/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
